### Import Libraries

In [ ]:
!pip uninstall vertexai google-cloud-aiplatform -y
!pip install google-cloud-aiplatform[preview]==1.94.0 -q

In [ ]:
# === BAGIAN 1: INSTALASI DAN OTENTIKASI ===

# Otentikasi pengguna ke Google Cloud
from google.colab import auth
auth.authenticate_user()

In [ ]:
import os
import json
import datetime
import re
import pandas as pd
from IPython.display import display, Markdown, HTML
from google import genai
from google.genai import types
from dotenv import load_dotenv
load_dotenv()

# Konfigurasi untuk Gemini API 2.0 (bukan Vertex AI)
GEMINI_API_KEY = os.getenv('gemini_api') # Ganti dengan API key Anda
NAMA_FILE_RISIKO_LOKAL = "SetiaModule/risk_data.json"

# Configure the client dengan API key (bukan Vertex AI)
client = genai.Client(api_key=GEMINI_API_KEY)

print("Setup awal selesai. Library ter-update dan API key dikonfigurasi.")

Setup awal selesai. Library ter-update dan API key dikonfigurasi.


### Main Functions

In [ ]:
# === SEL 2: FUNGSI-FUNGSI INTI ===
import re  # Tambahkan import re untuk regex

def get_risk_data_from_local(filepath):
    try:
        print(f"Membaca data risiko dari file lokal: {filepath}...")
        with open(filepath, 'r', encoding='utf-8') as f:
            risk_data = json.load(f)
        print("Berhasil membaca data risiko.")
        return risk_data
    except FileNotFoundError:
        print(f"\n❌ ERROR: File tidak ditemukan! Pastikan Anda sudah meng-upload '{filepath}' ke sesi Colab ini.")
        return None
    except Exception as e:
        print(f"\n❌ ERROR: Terjadi kesalahan saat membaca file lokal: {e}")
        return None

def get_grounded_news_analysis(company_name: str):
    """Memanggil model Gemini dengan grounding menggunakan Google GenAI SDK baru."""
    try:
        print(f"Meminta analisis AI untuk: {company_name}...")

        # Define the grounding tool menggunakan API baru
        grounding_tool = types.Tool(
            google_search=types.GoogleSearch()
        )

        # Configure generation settings
        config = types.GenerateContentConfig(
            tools=[grounding_tool]
        )

        prompt = f"""
        Cari dan analisis berita terkini tentang perusahaan "{company_name}" dalam Bahasa Indonesia.

        PENTING: Berikan respons HANYA dalam format berikut tanpa kata pengantar atau kalimat pembuka:

        RINGKASAN: [Tulis ringkasan konkret 2-3 kalimat tentang kondisi terkini perusahaan berdasarkan berita yang ditemukan]

        SENTIMEN: [Pilih: Positif/Negatif/Netral]

        ISU UTAMA:
        1. [Isu spesifik pertama dari berita]
        2. [Isu spesifik kedua dari berita]
        3. [Isu spesifik ketiga dari berita]

        Jangan gunakan kata pengantar seperti "Tentu, saya akan..." atau "Berikut adalah analisis...".
        Langsung mulai dengan "RINGKASAN:" dan berikan informasi faktual berdasarkan berita terkini.
        """

        # Make the request menggunakan Gemini API 2.0
        response = client.models.generate_content(
            model="gemini-2.0-flash-exp",  # Model Gemini 2.0
            contents=prompt,
            config=config
        )

        print("Respons AI berhasil diterima.")
        return response

    except Exception as e:
        print(f"\nERROR: Panggilan ke Gemini gagal. Error: {e}")
        return None

def process_and_display(raw_response, applicant_data, risk_data):
    """Mem-parsing respons, menambahkan sitasi, menampilkan, dan menyimpan."""
    try:
        applicant_name = applicant_data['applicant_name']

        # Extract text from new API response format
        if hasattr(raw_response, 'candidates') and raw_response.candidates:
            if hasattr(raw_response.candidates[0], 'content') and hasattr(raw_response.candidates[0].content, 'parts'):
                text = raw_response.candidates[0].content.parts[0].text
            else:
                # Fallback untuk format respons yang berbeda
                text = str(raw_response.candidates[0])
        else:
            # Fallback jika struktur berbeda
            text = str(raw_response)

        print(f"\n--- DEBUG: Teks mentah dari AI ---")
        print(text)
        print("--- END DEBUG ---\n")

        # Extract grounding metadata from new API response format
        grounding_metadata = None
        if hasattr(raw_response, 'candidates') and raw_response.candidates:
            if hasattr(raw_response.candidates[0], 'grounding_metadata'):
                grounding_metadata = raw_response.candidates[0].grounding_metadata

        # Extract sources from grounding chunks
        sources = []
        if grounding_metadata and hasattr(grounding_metadata, 'grounding_chunks') and grounding_metadata.grounding_chunks:
            for i, chunk in enumerate(grounding_metadata.grounding_chunks):
                sources.append({
                    "no": f"[{i+1}]",
                    "title": chunk.web.title if hasattr(chunk, 'web') and hasattr(chunk.web, 'title') else f"Source {i+1}",
                    "url": chunk.web.uri if hasattr(chunk, 'web') and hasattr(chunk.web, 'uri') else "N/A"
                })

        # Add citations using new metadata structure
        summary_with_citations = add_citations_from_metadata(text, grounding_metadata) if grounding_metadata else text

        # Extract sentiment from text dengan parsing yang lebih robust
        sentiment = extract_sentiment_from_text(text)

        # Extract key issues from formatted response
        key_issues = extract_key_issues_from_text(text)

        # Extract summary from formatted response
        summary = extract_summary_from_text(text)

        # Use extracted summary with citations if available, otherwise use full text with citations
        if summary and summary.strip():
            final_summary = add_citations_from_metadata(summary, grounding_metadata) if grounding_metadata else summary
        else:
            final_summary = summary_with_citations

        # Create final output structure
        final_output = {
            "applicantName": applicant_name,
            "groundedSummary": final_summary,
            "overallSentiment": sentiment,
            "keyIssues": key_issues,
            "supportingSources": sources,
            "industrySectorOutlook": risk_data.get("risiko_industri", {}).get(
                applicant_data.get('industry_main'), {}).get(
                applicant_data.get('industry_sub'), "Tidak Diketahui"),
            "analysisTimestamp": datetime.datetime.now(datetime.timezone.utc).isoformat()
        }

        display_results(final_output, applicant_name)
        save_result_to_json(final_output, applicant_name)

    except Exception as e:
        print(f"Gagal memproses hasil analisis. Error: {e}")
        print(f"\n--- Error Detail ---")
        import traceback
        traceback.print_exc()
        print("\n--- Teks Mentah dari AI ---")
        try:
            print(str(raw_response))
        except:
            print("Tidak dapat menampilkan teks mentah")

def extract_summary_from_text(text):
    """Extract summary from formatted AI response."""
    try:
        # Clean text first - remove citation markers
        cleaned_text = re.sub(r'\*\*\[\d+\]\*\*', '', text)

        # Look for "RINGKASAN:" section
        if "RINGKASAN:" in cleaned_text:
            lines = cleaned_text.split('\n')
            for i, line in enumerate(lines):
                if "RINGKASAN:" in line:
                    # Get the summary part after "RINGKASAN:"
                    summary_line = line.split("RINGKASAN:")[1].strip()
                    if summary_line:
                        return summary_line
                    # If summary is on next lines
                    elif i + 1 < len(lines):
                        next_lines = []
                        for j in range(i + 1, len(lines)):
                            next_line = lines[j].strip()
                            if next_line and not next_line.startswith("SENTIMEN:") and not next_line.startswith("ISU UTAMA:"):
                                next_lines.append(next_line)
                            else:
                                break
                        if next_lines:
                            return " ".join(next_lines)

        # Enhanced fallback - look for meaningful content patterns
        # Remove intro phrases that AI often uses
        intro_patterns = [
            r"Tentu, saya akan menganalisis.*?(?=\w)",
            r"Berikut adalah analisis.*?(?=\w)",
            r"Berdasarkan pencarian.*?(?=\w)",
            r"Saya akan memberikan.*?(?=\w)"
        ]

        for pattern in intro_patterns:
            cleaned_text = re.sub(pattern, '', cleaned_text, flags=re.IGNORECASE | re.DOTALL)

        # Split into sentences and find meaningful content
        sentences = re.split(r'[.!?]+', cleaned_text)
        meaningful_sentences = []

        for sentence in sentences:
            sentence = sentence.strip()
            # Skip very short sentences, intro phrases, and citation-only sentences
            if (len(sentence) > 20 and
                not sentence.lower().startswith(('tentu', 'saya akan', 'berikut', 'berdasarkan')) and
                not re.match(r'^\*\*\[\d+\]\*\*\s*') ): # Added closing parenthesis
                meaningful_sentences.append(sentence)

        if meaningful_sentences:
            # Take first 2-3 meaningful sentences as summary
            summary = '. '.join(meaningful_sentences[:3]) + '.'
            return summary

        # Final fallback - clean and return first part of text
        lines = [line.strip() for line in cleaned_text.split('\n') if line.strip()]
        if lines:
            first_meaningful_line = ""
            for line in lines:
                if len(line) > 20 and not line.lower().startswith(('tentu', 'saya akan', 'berikut')):
                    first_meaningful_line = line
                    break

            if first_meaningful_line:
                return first_meaningful_line

        return "Ringkasan tidak dapat diekstrak dari respons AI."

    except Exception as e:
        print(f"Error extracting summary: {e}")
        return "Error dalam mengekstrak ringkasan."

def extract_sentiment_from_text(text):
    """Extract sentiment from formatted AI response with improved parsing."""
    try:
        text_lower = text.lower()

        # Look for explicit sentiment declaration
        if "sentimen:" in text_lower:
            lines = text.split('\n')
            for line in lines:
                if "sentimen:" in line.lower():
                    sentiment_part = line.lower().split("sentimen:")[1].strip()
                    if "negatif" in sentiment_part:
                        return "Negatif"
                    elif "positif" in sentiment_part:
                        return "Positif"
                    elif "netral" in sentiment_part:
                        return "Netral"

        # Fallback sentiment analysis based on keywords
        negative_keywords = ["negatif", "turun", "merosot", "rugi", "kerugian", "masalah", "krisis", "penurunan", "buruk"]
        positive_keywords = ["positif", "naik", "meningkat", "untung", "keuntungan", "baik", "bagus", "tumbuh", "berkembang"]

        negative_count = sum(1 for keyword in negative_keywords if keyword in text_lower)
        positive_count = sum(1 for keyword in positive_keywords if keyword in text_lower)

        if positive_count > negative_count:
            return "Positif"
        elif negative_count > positive_count:
            return "Negatif"
        else:
            return "Netral"

    except Exception as e:
        print(f"Error extracting sentiment: {e}")
        return "Netral"

def extract_key_issues_from_text(text):
    """Extract key issues from formatted AI response with improved parsing."""
    key_issues = []
    try:
        # Look for "ISU UTAMA:" section
        if "ISU UTAMA:" in text:
            lines = text.split('\n')
            found_isu_section = False

            for line in lines:
                line = line.strip()

                if "ISU UTAMA:" in line:
                    found_isu_section = True
                    # Check if there's content after "ISU UTAMA:" on the same line
                    after_colon = line.split("ISU UTAMA:")[1].strip()
                    if after_colon and not after_colon.startswith("1."):
                        key_issues.append(after_colon)
                    continue

                if found_isu_section and line:
                    # Stop if we hit another section
                    if any(section in line.upper() for section in ["RINGKASAN:", "SENTIMEN:", "KESIMPULAN:"]):
                        break

                    # Extract numbered items
                    if line.startswith(('1.', '2.', '3.', '4.', '5.')):
                        issue = line[2:].strip()  # Remove "1. ", "2. ", etc.
                        if issue:
                            key_issues.append(issue)
                    # Extract bulleted items
                    elif line.startswith(('- ', '• ', '* ')):
                        issue = line[2:].strip()
                        if issue:
                            key_issues.append(issue)
                    # Extract plain text if we're in the ISU section
                    elif found_isu_section and not line.startswith(('RINGKASAN', 'SENTIMEN', 'ISU')):
                        if line and len(line) > 10:  # Reasonable length for an issue
                            key_issues.append(line)

        # Fallback: if no structured format found, try to extract from general text
        if not key_issues:
            # Look for patterns that might indicate issues
            lines = text.split('\n')
            for line in lines:
                line = line.strip()
                if line and any(keyword in line.lower() for keyword in ["masalah", "isu", "tantangan", "risiko", "kendala"]):
                    if len(line) > 10 and len(line) < 200:  # Reasonable length
                        key_issues.append(line)
                        if len(key_issues) >= 3:  # Limit to 3 issues
                            break

        # Final fallback
        if not key_issues:
            key_issues = ["Informasi isu belum tersedia dari analisis AI"]

    except Exception as e:
        print(f"Error extracting key issues: {e}")
        key_issues = ["Error dalam mengekstrak isu utama"]

    return key_issues[:3]  # Limit to maximum 3 issues

def add_citations_from_metadata(text, grounding_metadata):
    """Add citations to text using new API grounding metadata structure."""
    try:
        if not grounding_metadata or not hasattr(grounding_metadata, 'grounding_supports') or not grounding_metadata.grounding_supports:
            return text

        # Sort by end index in reverse order to avoid index shifting
        sorted_supports = sorted(
            grounding_metadata.grounding_supports,
            key=lambda s: s.segment.end_index,
            reverse=True
        )

        chunks = grounding_metadata.grounding_chunks

        for support in sorted_supports:
            end_index = support.segment.end_index
            if hasattr(support, 'grounding_chunk_indices') and support.grounding_chunk_indices:
                citation_links = []
                for i in support.grounding_chunk_indices:
                    if i < len(chunks):
                        citation_links.append(f" **[{i + 1}]**")

                # Insert citations at the end of the segment
                citation_text = "".join(citation_links)
                text = text[:end_index] + citation_text + text[end_index:]

        return text

    except Exception as e:
        print(f"Error adding citations: {e}")
        return text

def save_result_to_json(result_data, applicant_name):
    s_name = re.sub(r'[^\w\s-]', '', applicant_name).strip().replace(' ', '_')
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"hasil_setia_{s_name}_{timestamp}.json"
    output_dir = '/Output/Setia'

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    filepath = os.path.join(output_dir, filename)

    try:
        with open(filepath, 'w', encoding='utf-8') as f:
            json.dump(result_data, f, indent=2, ensure_ascii=False)
        print(f"\n✅ Hasil analisis berhasil disimpan ke file: {filepath}")
    except Exception as e:
        print(f"\n❌ Gagal menyimpan hasil ke file. Error: {e}")

def display_results(result_data, applicant_name):
    # Header dengan styling yang lebih menarik
    header_html = f"""
    <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                padding: 25px;
                border-radius: 15px;
                margin-bottom: 25px;
                box-shadow: 0 8px 32px rgba(0,0,0,0.1);">
        <h1 style="color: white;
                   text-align: center;
                   margin: 0;
                   font-size: 28px;
                   text-shadow: 2px 2px 4px rgba(0,0,0,0.3);">
            📊 HASIL ANALISIS SETIA
        </h1>
        <h2 style="color: #f8f9fa;
                   text-align: center;
                   margin: 8px 0 0 0;
                   font-weight: 300;
                   font-size: 20px;">
            {applicant_name}
        </h2>
    </div>
    """
    display(HTML(header_html))

    # Sentimen dengan card styling
    sentiment = result_data.get('overallSentiment', 'N/A')
    if sentiment == "Positif":
        sentiment_color = "#28a745"
        sentiment_bg = "#d4edda"
        sentiment_icon = "📈"
    elif sentiment == "Negatif":
        sentiment_color = "#dc3545"
        sentiment_bg = "#f8d7da"
        sentiment_icon = "📉"
    else:
        sentiment_color = "#6c757d"
        sentiment_bg = "#e2e3e5"
        sentiment_icon = "📊"

    sentiment_html = f"""
    <div style="background-color: {sentiment_bg};
                border-left: 5px solid {sentiment_color};
                padding: 20px;
                margin: 20px 0;
                border-radius: 8px;
                box-shadow: 0 2px 10px rgba(0,0,0,0.05);">
        <h3 style="color: {sentiment_color};
                   margin: 0 0 10px 0;
                   font-size: 22px;">
            {sentiment_icon} Sentimen Keseluruhan: <strong>{sentiment}</strong>
        </h3>
    </div>
    """
    display(HTML(sentiment_html))

    # Ringkasan AI dengan styling yang lebih baik
    summary = result_data.get('groundedSummary', 'Tidak ada ringkasan.')
    # Ensure summary is not empty or just citations
    if not summary.strip() or summary.strip() == 'Tidak ada ringkasan.':
        summary = "Ringkasan tidak tersedia dari analisis AI."

    summary_html = f"""
    <div style="background-color: #f8f9fa;
                border: 1px solid #dee2e6;
                padding: 20px;
                margin: 20px 0;
                border-radius: 10px;
                box-shadow: 0 2px 8px rgba(0,0,0,0.05);">
        <h3 style="color: #495057;
                   margin: 0 0 15px 0;
                   font-size: 18px;
                   border-bottom: 2px solid #007bff;
                   padding-bottom: 8px;">
            🤖 Ringkasan AI (Grounded)
        </h3>
        <div style="background-color: white;
                    padding: 15px;
                    border-radius: 6px;
                    border-left: 4px solid #007bff;
                    font-size: 15px;
                    line-height: 1.6;">
            {summary}
        </div>
    </div>
    """
    display(HTML(summary_html))

    # Isu-isu utama dengan styling list yang menarik
    key_issues = result_data.get('keyIssues', [])
    if key_issues:
        issues_html = """
        <div style="background-color: #fff3cd;
                    border: 1px solid #ffeaa7;
                    padding: 20px;
                    margin: 20px 0;
                    border-radius: 10px;
                    box-shadow: 0 2px 8px rgba(0,0,0,0.05);">
            <h3 style="color: #856404;
                       margin: 0 0 15px 0;
                       font-size: 18px;
                       border-bottom: 2px solid #ffc107;
                       padding-bottom: 8px;">
                ⚠️ Isu-Isu Utama
            </h3>
            <ul style="margin: 0; padding-left: 20px;">
        """
        for i, issue in enumerate(key_issues, 1):
            issues_html += f"""
                <li style="margin: 10px 0;
                           padding: 8px;
                           background-color: white;
                           border-radius: 5px;
                           border-left: 3px solid #ffc107;
                           font-size: 14px;">
                    <strong>#{i}:</strong> {issue}
                </li>
            """
        issues_html += "</ul></div>"
        display(HTML(issues_html))

    # Sumber berita dengan tabel yang lebih rapi
    sources = result_data.get('supportingSources')
    if sources:
        sources_html = """
        <div style="background-color: #e7f3ff;
                    border: 1px solid #b3d9ff;
                    padding: 20px;
                    margin: 20px 0;
                    border-radius: 10px;
                    box-shadow: 0 2px 8px rgba(0,0,0,0.05);">
            <h3 style="color: #004085;
                       margin: 0 0 15px 0;
                       font-size: 18px;
                       border-bottom: 2px solid #007bff;
                       padding-bottom: 8px;">
                🔗 Sumber Berita Pendukung (Referensi Sitasi)
            </h3>
        """

        # Create custom table HTML
        table_html = """
        <table style="width: 100%;
                      border-collapse: collapse;
                      background-color: white;
                      border-radius: 6px;
                      overflow: hidden;
                      box-shadow: 0 2px 4px rgba(0,0,0,0.1);">
            <thead>
                <tr style="background-color: #007bff; color: white;">
                    <th style="padding: 12px; text-align: center; width: 80px;">No</th>
                    <th style="padding: 12px; text-align: left;">Judul/Domain</th>
                    <th style="padding: 12px; text-align: center; width: 100px;">Link</th>
                </tr>
            </thead>
            <tbody>
        """

        for source in sources:
            # Extract domain from title if it looks like a domain
            title = source.get('title', 'N/A')
            if '.' in title and len(title.split('.')) <= 3:  # Likely a domain
                display_title = f"🌐 {title}"
            else:
                display_title = title

            table_html += f"""
                <tr style="border-bottom: 1px solid #dee2e6;">
                    <td style="padding: 12px;
                               text-align: center;
                               font-weight: bold;
                               color: #007bff;">
                        {source.get('no', '')}
                    </td>
                    <td style="padding: 12px;
                               font-size: 14px;">
                        {display_title}
                    </td>
                    <td style="padding: 12px;
                               text-align: center;">
                        <a href="{source.get('url', '#')}"
                           target="_blank"
                           style="background-color: #007bff;
                                  color: white;
                                  padding: 6px 12px;
                                  text-decoration: none;
                                  border-radius: 4px;
                                  font-size: 12px;
                                  display: inline-block;">
                            Buka 🔗
                        </a>
                    </td>
                </tr>
            """

        table_html += "</tbody></table>"
        sources_html += table_html + "</div>"
        display(HTML(sources_html))

    # Status lainnya dengan card info
    industry_outlook = result_data.get('industrySectorOutlook', 'Tidak Diketahui')
    timestamp = result_data.get('analysisTimestamp', 'N/A')

    # Format timestamp
    try:
        if timestamp != 'N/A':
            dt = datetime.datetime.fromisoformat(timestamp.replace('Z', '+00:00'))
            formatted_time = dt.strftime("%d %B %Y, %H:%M WIB")
        else:
            formatted_time = 'N/A'
    except:
        formatted_time = timestamp

    status_html = f"""
    <div style="background-color: #f8f9fa;
                border: 1px solid #dee2e6;
                padding: 20px;
                margin: 20px 0;
                border-radius: 10px;
                box-shadow: 0 2px 8px rgba(0,0,0,0.05);">
        <h3 style="color: #495057;
                   margin: 0 0 15px 0;
                   font-size: 18px;
                   border-bottom: 2px solid #6c757d;
                   padding-bottom: 8px;">
            📋 Informasi Tambahan
        </h3>
        <div style="display: grid;
                    grid-template-columns: 1fr 1fr;
                    gap: 15px;">
            <div style="background-color: white;
                        padding: 15px;
                        border-radius: 6px;
                        border-left: 4px solid #17a2b8;">
                <strong style="color: #17a2b8;">🏭 Prospek Sektor Industri:</strong><br>
                <span style="font-size: 14px; color: #6c757d;">{industry_outlook}</span>
            </div>
            <div style="background-color: white;
                        padding: 15px;
                        border-radius: 6px;
                        border-left: 4px solid #28a745;">
                <strong style="color: #28a745;">🕒 Waktu Analisis:</strong><br>
                <span style="font-size: 14px; color: #6c757d;">{formatted_time}</span>
            </div>
        </div>
    </div>
    """
    display(HTML(status_html))

print("Fungsi-fungsi inti siap digunakan.")


# === SEL 3: EKSEKUSI UTAMA ===

def main():
    print("\n" + "="*50)
    print("      MEMULAI SESI ANALISIS RISIKO SETIA (Google GenAI SDK)")
    print("="*50)
    print("\nSilakan masukkan detail aplikan di bawah ini.")

    applicant_name = input("➡️ Masukkan Nama Perusahaan/Aplikan: ")

    if applicant_name:
        applicant_data = {'applicant_name': applicant_name}
        applicant_data['industry_main'] = input("➡️ Masukkan Kategori Industri Utama (e.g., Keuangan): ")
        applicant_data['industry_sub'] = input("➡️ Masukkan Sub-Sektor Industri (e.g., Perbankan (BUKU IV/III)): ")

        risk_data = get_risk_data_from_local(NAMA_FILE_RISIKO_LOKAL)

        if risk_data:
            raw_response = get_grounded_news_analysis(applicant_name)
            if raw_response:
                display(Markdown("--- \n ## HASIL ANALISIS"))
                process_and_display(raw_response, applicant_data, risk_data)
            else:
                print("Tidak menerima respons dari AI, analisis dihentikan.")
        else:
            print("Data risiko tidak dapat dimuat, analisis dihentikan.")
    else:
        print("\nNama aplikan tidak diisi. Analisis dibatalkan.")

# Menjalankan program utama
main()

Fungsi-fungsi inti siap digunakan.

      MEMULAI SESI ANALISIS RISIKO SETIA (Google GenAI SDK)

Silakan masukkan detail aplikan di bawah ini.
➡️ Masukkan Nama Perusahaan/Aplikan: PT Alamtri Resources Indonesia Tbk
➡️ Masukkan Kategori Industri Utama (e.g., Keuangan): Pertambangan
➡️ Masukkan Sub-Sektor Industri (e.g., Perbankan (BUKU IV/III)): Batu Bara
Membaca data risiko dari file lokal: risk_data.json...
Berhasil membaca data risiko.
Meminta analisis AI untuk: PT Alamtri Resources Indonesia Tbk...
Respons AI berhasil diterima.


--- 
 ## HASIL ANALISIS


--- DEBUG: Teks mentah dari AI ---
RINGKASAN: Kinerja keuangan PT Alamtri Resources Indonesia Tbk (ADRO) mengalami penurunan pada kuartal I-2025, dengan laba bersih anjlok 79,5% dibandingkan periode yang sama tahun sebelumnya. Saham ADRO juga terpantau terus mengalami penurunan dalam sebulan terakhir, meskipun perusahaan meraup dividen dari anak usahanya.

SENTIMEN: Negatif

ISU UTAMA:
1.  Penurunan Laba Bersih: Laba bersih ADRO pada kuartal I-2025 merosot tajam sebesar 79,5% menjadi US$76,6 juta.
2.  Penurunan Pendapatan: Pendapatan usaha ADRO juga mengalami penurunan sebesar 22,3% menjadi US$381,6 juta pada kuartal I-2025.
3.  Penurunan Harga Saham: Harga saham ADRO terus mengalami penurunan, bahkan menyentuh level terendah dalam sebulan terakhir, meskipun ada pembagian dividen dari anak perusahaan.

--- END DEBUG ---



No,Judul/Domain,Link
[1],🌐 kontan.co.id,"<a href=""https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEyp0Ae7sRd6olIj-MSomRVHZo2pW7Y5pLdnE6IzWpFFFOvhj65gVYoRyiWvkZjyrdc43VQsrl5JTseWYGJBtkPrYuVQVKwX07Eri-kt-LuJQHZ4nP36eIGRf1FxoWd7UsOagBkrR3xg3Dqv4zdYxUJCNjSZQSMj3DmlmQyJJJa7trrnYavWxNQjd0QalrB8DmsUICsl5KBaeQB8bTxcsP9Eb-DvTrx2g=="" target=""_blank"" style=""background-color: #007bff; color: white; padding: 6px 12px; text-decoration: none; border-radius: 4px; font-size: 12px; display: inline-block;""> Buka 🔗"
[2],🌐 bisnis.com,"<a href=""https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQELig4_UbAeIElgvq8sPS4cMyg-c38hIr876q-lDofc08RfJGbSNHBEd4Y3jw9Exn1KQKM9lA-pzf2T5ywREmWp1_T2qtCjPCeYY_i2UYuiMTp2nx3ngB8AojKo6Tcrg7xUzR1PowUWGUhXasg3w4rz9GZ6x3tcpEsnMXaNYPjHdjrU_OVbRVtM3L7SaM3Yp3yrZ1Y5jA4sUp5emRblYHnsDrJX46laDoxfjx5uZFFEzN_p"" target=""_blank"" style=""background-color: #007bff; color: white; padding: 6px 12px; text-decoration: none; border-radius: 4px; font-size: 12px; display: inline-block;""> Buka 🔗"
[3],🌐 investor.id,"<a href=""https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFWUFliYExROUezzfyEtI3g-hSdxIQ3W4HMsoy42m1tHiZO__ojkBrBQIeHKTUnpmz_uosWoxls33zrcjFUnPoM1JXPrVHwFRpnz4W9gzGRZsbBrbtmH_oy9JW43duWxQFLCDkoSjRwD6cxs1wkpocVRQcx6lQKtbIEbhi4FLrFyuElgrROfpN5q5k="" target=""_blank"" style=""background-color: #007bff; color: white; padding: 6px 12px; text-decoration: none; border-radius: 4px; font-size: 12px; display: inline-block;""> Buka 🔗"



✅ Hasil analisis berhasil disimpan ke file: Output/Setia/hasil_setia_PT_Alamtri_Resources_Indonesia_Tbk_20250624_005100.json
